In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import hydrant.util.util as util
import pint

In [3]:
# Define dimensions
ID = 10 + np.arange(3)
seg = 20 + np.arange(5)
time = pd.date_range("2023-01-01", periods=4)  # Generating 4 time periods

# Create example data
data_vars = {}
fixed_values = [4, 7, 1]
data_vars['id'] = xr.DataArray(fixed_values, dims='ID')

# Creating variable1 with fixed values
data_vars['variable1'] = xr.DataArray(np.random.randint(1, 10, size=len(ID)), dims='ID')
# Creating variables 2 and 3 with specified units
data_vars['variable2'] = xr.DataArray(np.random.rand(len(seg), len(time)), dims=('seg', 'time'), attrs={'units': 'm**2'})
data_vars['variable3'] = xr.DataArray(np.random.rand(len(seg), len(ID)), dims=('seg', 'ID'), attrs={'units': 'K'})
data_vars['variable4'] = xr.DataArray(np.random.rand(len(time), len(seg), len(ID)), dims=('time', 'seg', 'ID'), attrs={'units': 'bar'})

# Create xarray dataset
ds = xr.Dataset(data_vars)

ds



<xarray.Dataset>
Dimensions:    (ID: 3, seg: 5, time: 4)
Dimensions without coordinates: ID, seg, time
Data variables:
    id         (ID) int64 4 7 1
    variable1  (ID) int64 9 5 3
    variable2  (seg, time) float64 0.03312 0.4326 0.7303 ... 0.2037 0.1929
    variable3  (seg, ID) float64 0.3149 0.01629 0.2966 ... 0.9135 0.1505 0.4999
    variable4  (time, seg, ID) float64 0.1598 0.7221 0.5441 ... 0.8441 0.9193

In [4]:
ds1 = util.drop_vars_with_dims(ds,
                         dims_to_drop = ['seg', 'time'])

print(ds1)

<xarray.Dataset>
Dimensions:    (ID: 3)
Dimensions without coordinates: ID
Data variables:
    id         (ID) int64 4 7 1
    variable1  (ID) int64 9 5 3


In [5]:
ds2 = util.drop_vars_with_dims(ds,
                         dims_to_drop = ['seg'])

print(ds2)

<xarray.Dataset>
Dimensions:    (ID: 3)
Dimensions without coordinates: ID
Data variables:
    id         (ID) int64 4 7 1
    variable1  (ID) int64 9 5 3


In [6]:
ds3 = util.keep_vars_with_dims(ds,
                        dims_to_keep = ['seg'])
print(ds3)

<xarray.Dataset>
Dimensions:    (seg: 5, time: 4, ID: 3)
Dimensions without coordinates: seg, time, ID
Data variables:
    variable2  (seg, time) float64 0.03312 0.4326 0.7303 ... 0.2037 0.1929
    variable3  (seg, ID) float64 0.3149 0.01629 0.2966 ... 0.9135 0.1505 0.4999
    variable4  (time, seg, ID) float64 0.1598 0.7221 0.5441 ... 0.8441 0.9193


In [7]:
ds4 = util.vars_to_keep(ds,
                        vars_to_keep = ['variable2','variable1'])
print(ds4)

<xarray.Dataset>
Dimensions:    (seg: 5, time: 4, ID: 3)
Dimensions without coordinates: seg, time, ID
Data variables:
    variable2  (seg, time) float64 0.03312 0.4326 0.7303 ... 0.2037 0.1929
    variable1  (ID) int64 9 5 3


In [8]:
print(ds['ID'])
print(ds['variable3'])

ds5 = util.reorder(ds,
                   [7,1],
                   mapping = {'var_id':'id','dim_id':'ID'})

print(ds5['ID'])
print(ds5['variable3'])
ds5

<xarray.DataArray 'ID' (ID: 3)>
array([0, 1, 2])
Dimensions without coordinates: ID
<xarray.DataArray 'variable3' (seg: 5, ID: 3)>
array([[0.31494158, 0.01629267, 0.29658876],
       [0.638672  , 0.4672656 , 0.1748959 ],
       [0.55082652, 0.35072865, 0.34514485],
       [0.6204037 , 0.9404583 , 0.98940508],
       [0.91347631, 0.15050215, 0.49993527]])
Dimensions without coordinates: seg, ID
Attributes:
    units:    K
<xarray.DataArray 'ID' (ID: 2)>
array([0, 1])
Dimensions without coordinates: ID
<xarray.DataArray 'variable3' (seg: 5, ID: 2)>
array([[0.01629267, 0.29658876],
       [0.4672656 , 0.1748959 ],
       [0.35072865, 0.34514485],
       [0.9404583 , 0.98940508],
       [0.15050215, 0.49993527]])
Dimensions without coordinates: seg, ID
Attributes:
    units:    K


<xarray.Dataset>
Dimensions:    (ID: 2, seg: 5, time: 4)
Dimensions without coordinates: ID, seg, time
Data variables:
    id         (ID) int64 7 1
    variable1  (ID) int64 5 3
    variable2  (seg, time) float64 0.03312 0.4326 0.7303 ... 0.2037 0.1929
    variable3  (seg, ID) float64 0.01629 0.2966 0.4673 ... 0.9894 0.1505 0.4999
    variable4  (time, seg, ID) float64 0.7221 0.5441 0.6101 ... 0.8441 0.9193

In [9]:
print(ds['variable4'])

ds6 = util.custom_sum(ds,\
                      weight=None, \
                      dims={'dim_time': 'time', 'dim_id': 'ID'})

print(ds6['variable4'])

<xarray.DataArray 'variable4' (time: 4, seg: 5, ID: 3)>
array([[[0.15978569, 0.72206546, 0.54408795],
        [0.05816262, 0.61005345, 0.61344109],
        [0.29065703, 0.94419052, 0.59131745],
        [0.46399429, 0.80519701, 0.02777839],
        [0.45541561, 0.62846446, 0.25944863]],

       [[0.27112489, 0.81528914, 0.36722374],
        [0.06855799, 0.98195167, 0.91948278],
        [0.36085651, 0.55622868, 0.600923  ],
        [0.59267354, 0.29971831, 0.49059325],
        [0.43143438, 0.6999031 , 0.7647871 ]],

       [[0.79231897, 0.4936127 , 0.66129867],
        [0.02049746, 0.4744123 , 0.88304097],
        [0.74221827, 0.91489953, 0.22075339],
        [0.38059722, 0.29767957, 0.87887813],
        [0.22630646, 0.20393939, 0.30271829]],

       [[0.9892215 , 0.74376158, 0.56922224],
        [0.24635523, 0.91785672, 0.04934224],
        [0.54556771, 0.08183918, 0.11439745],
        [0.17936999, 0.50155368, 0.28250818],
        [0.12241214, 0.84412929, 0.91934165]]])
Dimensions witho

In [10]:
print(ds['ID'])
print(ds['variable3'])

ds7 = util.reorder(ds,
                   [7,1],
                   mapping = {'var_id':'id','dim_id':'ID'})

print(ds7['ID'])
print(ds7['variable3'])
ds7

<xarray.DataArray 'ID' (ID: 3)>
array([0, 1, 2])
Dimensions without coordinates: ID
<xarray.DataArray 'variable3' (seg: 5, ID: 3)>
array([[0.31494158, 0.01629267, 0.29658876],
       [0.638672  , 0.4672656 , 0.1748959 ],
       [0.55082652, 0.35072865, 0.34514485],
       [0.6204037 , 0.9404583 , 0.98940508],
       [0.91347631, 0.15050215, 0.49993527]])
Dimensions without coordinates: seg, ID
Attributes:
    units:    K
<xarray.DataArray 'ID' (ID: 2)>
array([0, 1])
Dimensions without coordinates: ID
<xarray.DataArray 'variable3' (seg: 5, ID: 2)>
array([[0.01629267, 0.29658876],
       [0.4672656 , 0.1748959 ],
       [0.35072865, 0.34514485],
       [0.9404583 , 0.98940508],
       [0.15050215, 0.49993527]])
Dimensions without coordinates: seg, ID
Attributes:
    units:    K


<xarray.Dataset>
Dimensions:    (ID: 2, seg: 5, time: 4)
Dimensions without coordinates: ID, seg, time
Data variables:
    id         (ID) int64 7 1
    variable1  (ID) int64 5 3
    variable2  (seg, time) float64 0.03312 0.4326 0.7303 ... 0.2037 0.1929
    variable3  (seg, ID) float64 0.01629 0.2966 0.4673 ... 0.9894 0.1505 0.4999
    variable4  (time, seg, ID) float64 0.7221 0.5441 0.6101 ... 0.8441 0.9193

In [13]:
# set the unit registery for millibar
_ureg = pint.UnitRegistry(force_ndarray_like=True)
_ureg.define('millibar = 1e-3 * bar')

ds8 = util.convert_units(ds,
                         units = {'variable1':'m**2','variable2':'kelvin','variable4':'bar'},
                         unit_registry = _ureg,
                         to_units= {'variable1':'km**2','variable2':'degC','variable4':'millibar'})

ds8

<xarray.Dataset>
Dimensions:    (ID: 3, seg: 5, time: 4)
Dimensions without coordinates: ID, seg, time
Data variables:
    id         (ID) int64 4 7 1
    variable1  (ID) float64 9e-06 5e-06 3e-06
    variable2  (seg, time) float64 -273.1 -272.7 -272.4 ... -272.5 -272.9 -273.0
    variable3  (seg, ID) float64 0.3149 0.01629 0.2966 ... 0.9135 0.1505 0.4999
    variable4  (time, seg, ID) float64 159.8 722.1 544.1 ... 122.4 844.1 919.3